# 음성 메시지를 텍스트로 변환

최첨단 음성-텍스트 기능을 위한 OpenAI의 Whisper API와 정확한 텍스트 번역을 위한 ChatCompletion API를 활용하여 모든 음성 메시지를 선택한 언어로 번역할 수 있는 엔드투엔드 솔루션 작성.

- **음성 인식용 Whisper API**: 음성 메시지를 텍스트로 변환하는 OpenAI의 Whisper API



In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [2]:
from openai import OpenAI
client = OpenAI()

# Whisper로 오디오 파일 처리하기

### 오디오 전사(Audio transcription)

- 오디오가 어떤 언어로 되어 있든 오디오를 전사할 수 있습니다.  

#### respose format
- vtt(Web Video Text Tracks) 형식 - 비디오 자막에 주로 사용되며, 시간 코드와 함께 텍스트 정보를 포함  
- text - TEXT 형식

In [3]:
# 오디오 파일을 바이너리 읽기 모드로 열기
audio_file = open("data/voice_1.m4a", "rb") 

# 오디오 파일을 전사하기 위한 요청
transcript = client.audio.transcriptions.create(
  model="whisper-1",        # 사용할 전사 모델 지정
  file=audio_file,                # 전사할 오디오 파일
  response_format='vtt'     # 전사 결과의 형식 지정 - vtt 또는 text
)

In [4]:
print(transcript)

WEBVTT

00:00:00.000 --> 00:00:10.000
한국의 자연은 울창한 살림과 아름다운 해안선, 그리고 계절에 따라 변하는 매혹적인 경관으로 잘 알려져 있습니다.

00:00:10.000 --> 00:00:19.000
봄에는 전국에 피어나는 화려한 벚꽃이 매력적인 장관을 연출하고, 가을에는 단풍이 산과 공원을 물들입니다.

00:00:19.000 --> 00:00:25.000
나는 우리나라를, 그리고 우리나라의 산천을 정말로 사랑합니다.




### 오디오 번역(Audio translation)

- 오디오를 영어로 번역하고 기록합니다.

In [5]:
# 오디오 파일 번역 요청
transcript_translated = client.audio.translations.create(
  model="whisper-1",        # 사용할 번역 모델 지정 (여기서는 'whisper-1')
  file=audio_file           # 번역할 오디오 파일
)

# 번역된 결과 출력
transcript_translated.text

'The nature of Korea is well known for its lush forests, beautiful coastlines, and fascinating landscapes that change according to the season. In spring, the colorful cherry blossoms that bloom all over the country create an attractive landscape, and in autumn, the leaves color the mountains and parks. I really love our country and our mountains and rivers.'

### Auido를 원하는 언어로 번역

`whisper-1`은 audio를 영어로 번역하므로 영어가 아닌 언어를 target으로 할 경우 ChatGpt 모델과 연결하여 번역

In [6]:
# 번역 대상 언어 설정
target_language = "일본어"

# AI의 역할을 메시지 리스트에 추가
messages = [
    {
        "role": "developer",
        "content": """나는 당신이 {} 로 번역하는 알고리즘 역할을 하길 원합니다.  
                      사용자가 텍스트를 제공하면, 당신의 역할은 이를 {} 로 번역하는 것입니다.  
                      """.format(target_language, target_language)
    }
]
# 번역할 텍스트를 메시지 리스트에 추가
messages.append({"role": "user", "content": transcript_translated.text})

# 대화형 AI로부터 번역 응답 생성 요청
chat_response = client.chat.completions.create(
    model="gpt-4o-mini",        # 대화형 AI 모델 지정
    messages=messages,      # 대화 내용
    temperature=0.9,              # 창의성을 위한 온도 설정
    max_tokens=2000,            # 생성할 최대 토큰 수
)

In [7]:
print("Assistant:", chat_response.choices[0].message.content)

Assistant: 韓国の自然は、その豊かな森林、美しい海岸線、季節ごとに変わる魅力的な風景でよく知られています。春には、全国各地に咲き誇るカラフルな桜が美しい景観を作り、秋には、山々や公園の葉が色づきます。私は本当に私たちの国や山や川を愛しています。


### GPT-4 출력을 Audio speech 로 변환


In [8]:
import warnings
warnings.filterwarnings('ignore')

Model = "tts-1"

speech_file_path = "output/vision_speech.mp3"

audio_response = client.audio.speech.create(
  model=Model,
  voice="alloy",
  input=chat_response.choices[0].message.content
)

audio_response.stream_to_file(speech_file_path)

In [9]:
from IPython.display import Audio

# 생성된 음성을 파일로 저장
audio_response.stream_to_file(speech_file_path)

# 저장된 음성 파일을 자동 재생으로 출력
Audio(speech_file_path, autoplay=True)